<a href="https://colab.research.google.com/github/AnupamaRajkumar/AppliedDeepLearning/blob/master/Object_detection_on_VOC12_cars_and_motorbikes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1/C
**Disclaimer: Only for ADL/AML students!**

### General information
**You have to solve all tasks to pass!** 

Grade is calculated by the day of the last submission, but you will only get it after you've succesfully presented it. 

**Deadlines and grades:** 
  * 2020.09.20 - 2020.10.27 ==> 5
  * 2020.10.28 - 2020.11.03 ==> 4
  * 2020.10.04 - 2020.11.10 ==> 3
  * 2020.11.11 - 2020.11.17 ==> 2
  * 2020.11.18 or later ==> 1 

You can **use only these** 3rd party **packages:** `cv2, keras, matplotlib, numpy, sklearn, skimage, tensorflow`. (And `torchvision` for dowloading the dataset.)

### Description
In this assignment you have to build and train a simple car and motorbike detection model using tf.keras. To do so, first we crop the images, the classify each crop one-by-one. For architecture we will use a simple convolutional network.

### Use GPU
Runtime -> Change runtime type

At Hardware accelerator select  GPU then save it.  

### Useful shortcuts
* Run selected cell: *Ctrl + Enter*
* Insert cell below: *Ctrl + M B*
* Insert cell above: *Ctrl + M A*
* Convert to text: *Ctrl + M M*
* Split at cursor: *Ctrl + M -*
* Autocomplete: *Ctrl + Space* or *Tab*
* Move selected cells up: *Ctrl + M J*
* Move selected cells down: *Ctrl + M K*
* Delete selected cells: *Ctrl + M D*



## Prepare dataset

* Download the PascalVOC2012 detection dataset. Here you can find more information about the labels: http://host.robots.ox.ac.uk/pascal/VOC/voc2012/htmldoc/index.html or https://pjreddie.com/media/files/VOC2012_doc.pdf *Hint: you don't need to load the data from scrach, you can iterate over the `data` variable, it will automatically read the image and the label.*
* To see some samples, plot 5 random images with their corresponding bounding boxes (bboxes). *Note: it's enough if you just print the bboxes, but you can also visualize them using `cv2.rectangle()`.* 
* Take an NxN sized crop from the center of each image, where `N = min(height, width); if N % 2 != 0: N -= 1`.
* Select only those, that contains cars or motorbikes.
* Split the image into 9 overlapping regions with a crop size of `<img_size> // 2` and a stride of `<img_size> // 4`, and classify each of them into 3 categories: `0 -- background, 1 -- motorbike, 2 -- car`.
  * if there isn't any (car or motorbike) bbox that overlaps more than by 50% with the crop, then it is a background crop.
  * otherwise it is a car/motorbike crop.
* Then resize the cropped regions to 64x64.
* Split the datasets into train-val-test sets (ratio: 60-20-20), without shuffling.
* Print the size of each set and plot 5 training images and their corresponding masks.
* Normalize the datasets. Input values should be between -1.0 and 1.0. *Note: you don't have to use standardization, you can just divide them by 255.*


In [ ]:
#@title
%%html
<p><span style="font-family: sans-serif; font-size: 1rem; font-style: italic">Note: Stride is a step size. For example, if you have a 4x4 image, and you'd like to split it into 9 regions with a crop size of 2 and a stride of 1, then first you'd take the top left 2x2 region</span></p>
<table style="height: 104px; width: 30%; border-collapse: collapse;" border="1">
<tbody>
<tr style="height: 26px;">
<td style="width: 25%; height: 26px; background: #00f;"></td>
<td style="width: 25%; height: 26px; background: #00f;"></td>
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
</tr>
<tr style="height: 26px;">
<td style="width: 25%; height: 26px; background: #00f;"></td>
<td style="width: 25%; height: 26px; background: #00f;"></td>
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
</tr>
<tr style="height: 26px;">
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
</tr>
<tr style="height: 26px;">
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
</tr>
</tbody>
</table>
<p><span style="font-family: sans-serif; font-size: 1rem; font-style: italic">Then the top middle 2x2 region</span></p>
<table style="height: 104px; width: 30%; border-collapse: collapse;" border="1">
<tbody>
<tr style="height: 26px;">
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px; background: #00f;"></td>
<td style="width: 25%; height: 26px; background: #00f;"></td>
<td style="width: 25%; height: 26px;"></td>
</tr>
<tr style="height: 26px;">
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px; background: #00f;"></td>
<td style="width: 25%; height: 26px; background: #00f;"></td>
<td style="width: 25%; height: 26px;"></td>
</tr>
<tr style="height: 26px;">
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
</tr>
<tr style="height: 26px;">
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
</tr>
</tbody>
</table>
<p><span style="font-family: sans-serif; font-size: 1rem; font-style: italic">Then the top right 2x2 region.</span></p>
<table style="height: 104px; width: 30%; border-collapse: collapse;" border="1">
<tbody>
<tr style="height: 26px;">
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px; background: #00f;"></td>
<td style="width: 25%; height: 26px; background: #00f;"></td>
</tr>
<tr style="height: 26px;">
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px; background: #00f;"></td>
<td style="width: 25%; height: 26px; background: #00f;"></td>
</tr>
<tr style="height: 26px;">
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
</tr>
<tr style="height: 26px;">
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
</tr>
</tbody>
</table>
<p><span style="font-family: sans-serif; font-size: 1rem; font-style: italic">Then the middle left 2x2 region.</span></p>
<table style="height: 104px; width: 30%; border-collapse: collapse;" border="1">
<tbody>
<tr style="height: 26px;">
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
</tr>
<tr style="height: 26px;">
<td style="width: 25%; height: 26px; background: #00f;"></td>
<td style="width: 25%; height: 26px; background: #00f;"></td>
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
</tr>
<tr style="height: 26px;">
<td style="width: 25%; height: 26px; background: #00f;"></td>
<td style="width: 25%; height: 26px; background: #00f;"></td>
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
</tr>
<tr style="height: 26px;">
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
<td style="width: 25%; height: 26px;"></td>
</tr>
</tbody>
</table>
<p><span style="font-family: sans-serif; font-size: 1rem; font-style: italic">And so on...</span></p>

,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,


In [ ]:
pip install torch===1.4.0 torchvision===0.5.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!mkdir -p ./voc2012

# Download from Drive
!if ! [ -f ./voc2012/VOCtrainval_11-May-2012.tar ]; then curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=19Mh6P8sXJzD_j0O2AN_StB2fbqBXJNWA" > /dev/null; curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=19Mh6P8sXJzD_j0O2AN_StB2fbqBXJNWA" -o ./voc2012/VOCtrainval_11-May-2012.tar; fi

# Check if the file size is correct (~1.86GB)
!if (( $(stat -c%s ./voc2012/VOCtrainval_11-May-2012.tar) < 1999639040 )); then rm -rfd ./voc2012/VOCtrainval_11-May-2012.tar; fi

# If not, download it from NIPG12
!wget -nc -O ./voc2012/VOCtrainval_11-May-2012.tar http://nipg1.inf.elte.hu:8000/VOCtrainval_11-May-2012.tar

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import skimage
import sklearn
import torchvision

data = torchvision.datasets.VOCDetection('./voc2012/', year='2012', image_set='trainval', download=True)

## Data augmentation
  * Augment the training set using `ImageDataGenerator`. The parameters should be the following: `featurewise_center=False, featurewise_std_normalization=False, rotation_range=90., width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.2, horizontal_flip=True`.
  * When creating the generator(s), use shuffling with a seed value of 1 and batch size of 32.
  * To validate that the augmentation is working, plot 5 original images with their corresponding transformed (augmented) images and labels.


## Define the model
Define the following convolutional network:
* It has 6 conv layers with 3x3 kernes
* Filter sizes are 32, 32, 64, 64, 256, 256 (in this order)
* The 1., 3. and 5. layers are padded such that the output is the same as the input
* After every convolution comes a BatchNormalisation and ReLU activation layer
* After every second convolution comes a MaxPool layer with 2x2 pool size and a Dropout layer with rate 20%
* After the last convolutional layer add a GlobalAveragePooling2D layer and a final dense layer
* For optimizer use Adam, and add accuracy to the metrics. *Note: For multi-class classification tasks, use cross-entropy loss.*

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *

In [ ]:
def classnet(input_size=(64, 64, 3)):
  pass

model = classnet(input_size=(64, 64, 3))
model.summary()

## Training and evaluation 
  * Train the model for 50 epochs and use early stopping with patience of 10.
  * Plot the training curve (train/validation loss and train/validation accuracy).
  * Evaluate the trained model on the test set and plot some (5) predictions with their corresponding GT labels.
  * Print the classification report and the multilabel confusion matrix. *Note: check out the following functions: `sklearn.metrics.classification_report` and `sklearn.metrics.multilabel_confusion_matrix`*
